# Laboratorio 2: Armado de un esquema de aprendizaje automático

En el laboratorio final se espera que puedan poner en práctica los conocimientos adquiridos en el curso, trabajando con un conjunto de datos de clasificación.

El objetivo es que se introduzcan en el desarrollo de un esquema para hacer tareas de aprendizaje automático: selección de un modelo, ajuste de hiperparámetros y evaluación.

El conjunto de datos a utilizar está en `./data/loan_data.csv`. Si abren el archivo verán que al principio (las líneas que empiezan con `#`) describen el conjunto de datos y sus atributos (incluyendo el atributo de etiqueta o clase).

Se espera que hagan uso de las herramientas vistas en el curso. Se espera que hagan uso especialmente de las herramientas brindadas por `scikit-learn`.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## Carga de datos y división en entrenamiento y evaluación

La celda siguiente se encarga de la carga de datos (haciendo uso de pandas). Estos serán los que se trabajarán en el resto del laboratorio.

In [2]:
dataset = pd.read_csv("./data/loan_data.csv", comment="#")

# División entre instancias y etiquetas
X, y = dataset.iloc[:, 1:], dataset.TARGET

# división entre entrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


Documentación:

- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

## Ejercicio 1: Descripción de los Datos y la Tarea

Responder las siguientes preguntas:

1. ¿De qué se trata el conjunto de datos?
2. ¿Cuál es la variable objetivo que hay que predecir? ¿Qué significado tiene?
3. ¿Qué información (atributos) hay disponible para hacer la predicción?
4. ¿Qué atributos imagina ud. que son los más determinantes para la predicción?

**No hace falta escribir código para responder estas preguntas.**

1 - The dataset consists of a data related to banking and loan review. 

2 - The variable ```TARGET``` is a boolean that identifies if the prospective bank lender has thier loan approved or not. The value ```1``` represents approved and ```0``` not approved. 

3 - The features available for this are things such as the mortgage, the value of the loan ```LOAN```, whether the lender has been delinquent ```DELINQ```, debt incurred ```DEBTINC```. 

In [3]:
selector = (dataset['TARGET'] == 0)
print("PERCENT DENIED:",dataset[selector].shape[0] / dataset.shape[0])

PERCENT DENIED: 0.8333333333333334


In [4]:
features = dataset.columns.to_list()
features.remove('TARGET')
print(features)

['LOAN', 'MORTDUE', 'VALUE', 'YOJ', 'DEROG', 'DELINQ', 'CLAGE', 'NINQ', 'CLNO', 'DEBTINC']


4 - We guess that the size of the loan ```LOAN``` and the past delinquency ```DEBTINC``` will affect the outcome of the loan application. 

## Ejercicio 2: Predicción con Modelos Lineales

En este ejercicio se entrenarán modelos lineales de clasificación para predecir la variable objetivo.

Para ello, deberán utilizar la clase SGDClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/sgd.html
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html


### Ejercicio 2.1: SGDClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador SGDClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

In [5]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV

In [6]:
clf = SGDClassifier(random_state=4)#semilla
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
print("ACC:",accuracy_score(y_test, y_pred))
print("PREC",precision_score(y_test, y_pred))
print("RECALL",recall_score(y_test, y_pred))
print("F1",f1_score(y_test, y_pred)) 
print('CONFUSION MATRIX:')
confusion_matrix(y_test, y_pred) # there are NO true negatives!

ACC: 0.8355795148247979
PREC 0.0
RECALL 0.0
F1 0.0
CONFUSION MATRIX:


array([[310,   3],
       [ 58,   0]])

![confusion matrix](images/confusion_matrix.png "confusion matrix")

There is a problem: Because there are so few training examples it is difficult to have a meaningful model. 

### Ejercicio 2.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del SGDClassifier. Como mínimo, probar diferentes funciones de loss, tasas de entrenamiento y tasas de regularización.

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [8]:
from sklearn.model_selection import cross_val_score
print("Accuracy of each fold:",cross_val_score(clf, X, y, cv=5))
print("Mean accuracy of all of the folds:",cross_val_score(clf, X, y, cv=5).mean())
#print(cross_val_score(clf, X, y, cv=5).var())
print("Standard Deviation of accuracies:",cross_val_score(clf, X, y, cv=5).std())

Accuracy of each fold: [0.82210243 0.8328841  0.8328841  0.74393531 0.82972973]
Mean accuracy of all of the folds: 0.8123071319297734
Standard Deviation of accuracies: 0.03441227606875618


## Grid Search hyperparameters

In [9]:
param_grid = {
    'loss' : ['hinge', 'log', 'perceptron', 'modified_huber', 'squared_hinge'],
    'penalty' : ['l2', 'l1', 'elasticnet'], 
    'learning_rate' : ['constant','optimal','invscaling','adaptive'],
    'eta0' : [0.1,0.2,0.4,1]
}

In [10]:
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(clf, param_grid, scoring='accuracy', cv=3)
cv.fit(X, y);

In [11]:
results = cv.cv_results_
df = pd.DataFrame(results)
df[['param_loss', 'param_penalty','param_eta0', 'mean_test_score', 'std_test_score', 'rank_test_score']].head()

,param_loss,param_penalty,param_eta0,mean_test_score,std_test_score,rank_test_score
0,hinge,l2,0.1,0.780475,0.074753,138
1,hinge,l1,0.1,0.833333,0.000000,1
2,hinge,elasticnet,0.1,0.831176,0.003051,30
3,log,l2,0.1,0.831176,0.003051,30
4,log,l1,0.1,0.805286,0.027151,121


In [12]:
## find the average and variance of all of the scores
cv.best_params_

{'eta0': 0.1, 'learning_rate': 'constant', 'loss': 'hinge', 'penalty': 'l1'}

In [82]:
# for the best configuration report
clf = SGDClassifier(random_state=4, eta0= 0.1, learning_rate='constant',loss='hinge',penalty='l1')#semilla
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)
print("ACC:",accuracy_score(y_test, y_pred).round(2))
print("PREC",precision_score(y_test, y_pred).round(2))
print("RECALL",recall_score(y_test, y_pred).round(2))
print("F1",f1_score(y_test, y_pred).round(2)) # Voila! magically we now have precision and recall!
print('CONFUSION MATRIX:')
confusion_matrix(y_test, y_pred)

ACC: 0.82
PREC 0.17
RECALL 0.03
F1 0.06
CONFUSION MATRIX:


array([[303,  10],
       [ 56,   2]])

Commentary:
    Still no precision or recall but the accuracy has increased marginally after hyperparameter optimization. 

![confusion matrix](images/confusion_matrix.png "confusion matrix")

## Ejercicio 3: Árboles de Decisión

En este ejercicio se entrenarán árboles de decisión para predecir la variable objetivo.

Para ello, deberán utilizar la clase DecisionTreeClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/tree.html
  - https://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
- https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html

### Ejercicio 3.1: DecisionTreeClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador DecisionTreeClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


In [14]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

In [15]:
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)
prec_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
print(f'Train accuracy: {train_acc:0.2}')
print(f'Test accuracy : {test_acc:0.2}')
print(f'Test precision: {prec_test:0.2}')
print(f'Test recall   : {recall_test:0.2}')
print(f'Test f1 score : {f1_test:0.2}')
print('CONFUSION MATRIX:')
confusion_matrix(y_test, y_pred)

Train accuracy: 1.0
Test accuracy : 0.88
Test precision: 0.62
Test recall   : 0.64
Test f1 score : 0.63
CONFUSION MATRIX:


array([[310,   3],
       [ 58,   0]])

![confusion matrix](images/confusion_matrix.png "confusion matrix")

### Ejercicio 3.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del DecisionTreeClassifier. Como mínimo, probar diferentes criterios de partición (criterion), profundidad máxima del árbol (max_depth), y cantidad mínima de samples por hoja (min_samples_leaf).

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [16]:
#kf = KFold(n_splits=5, shuffle=False, random_state=0)
print("Accuracy of each fold:",cross_val_score(clf, X, y, cv=5))
print("Mean accuracy of all of the folds:",cross_val_score(clf, X, y, cv=5).mean())
#print(cross_val_score(clf, X, y, cv=5).var())
print("Standard Deviation of accuracies:",cross_val_score(clf, X, y, cv=5).std())

Accuracy of each fold: [0.8328841  0.8328841  0.8490566  0.86522911 0.87297297]
Mean accuracy of all of the folds: 0.8506053762657537
Standard Deviation of accuracies: 0.016399080200692626


In [17]:
param_grid = {
    'min_samples_leaf' : [1,2,3,4,5,6,7,8,9,10],
    'criterion' : ['gini', 'entropy'], 
    'max_depth' : [1,2,3]
}

In [18]:
clf = DecisionTreeClassifier(random_state=0)

In [19]:
cv = GridSearchCV(clf, param_grid, scoring='accuracy', cv=3)
cv.fit(X, y);

In [20]:
results = cv.cv_results_
df = pd.DataFrame(results)
df[['param_max_depth', 'param_criterion','param_min_samples_leaf', 'mean_test_score', 'std_test_score', 'rank_test_score']].head()

,param_max_depth,param_criterion,param_min_samples_leaf,mean_test_score,std_test_score,rank_test_score
0,1,gini,1,0.87109,0.013234,7
1,1,gini,2,0.87109,0.013234,7
2,1,gini,3,0.87109,0.013234,7
3,1,gini,4,0.87109,0.013234,7
4,1,gini,5,0.87109,0.013234,7


In [21]:
cv.best_params_

{'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 9}

In [118]:
clf = DecisionTreeClassifier(criterion='gini', max_depth=2, min_samples_leaf=9,random_state=0)
clf.fit(X_train, y_train)
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)
prec_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
print(f'Train accuracy: {train_acc:0.2}')
print(f'Test accuracy : {test_acc:0.2}')
print(f'Test precision: {prec_test:0.2}')
print(f'Test recall   : {recall_test:0.2}')
print(f'Test f1 score : {f1_test:0.2}')
print('CONFUSION MATRIX:')
confusion_matrix(y_test, y_pred)


Train accuracy: 0.87
Test accuracy : 0.88
Test precision: 0.85
Test recall   : 0.29
Test f1 score : 0.44
CONFUSION MATRIX:


array([[310,   3],
       [ 58,   0]])

### Ejercicio 3.3: Inspección del Modelo

The challenge of this dataset was that the binary target feature, approval or disapproval of a loan, was greatly weighted towards disapproval; less than 20 percent of the training samples were of loan cases that were approved. In cases like this a data scientist runs the risk of building a model that always gives the same answer, the category that has the majority of the "votes" from the training data. In that way, this example is good to show the advantages and, in some cases, necessities to perform the tecniques Grid Search and Cross Validation as a preparation for designing the architeqcture and training specifications for your machine learning model, especially in the case of classification data that is imbalanced or weighted particularly towards one class. 